# 패키지불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install colormath
!pip install tensorflow-addons


In [ ]:
import asyncio
import datetime
import logging
import traceback

import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.metrics import r2_score
import pickle
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from colormath.color_objects import LabColor, sRGBColor, XYZColor, LabColor
from colormath.color_diff import delta_e_cmc
from colormath.color_conversions import convert_color

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors

from PIL import Image
import math
from math import sqrt
import copy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
import pandas.core.algorithms as algos
from pandas import Series
import pickle
#from pandas import tools

In [ ]:
import warnings

# 모든 경고 무시
warnings.filterwarnings("ignore")

In [ ]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cmc
import numpy as np
import pandas as pd
from colormath.color_objects import sRGBColor, XYZColor, LabColor
from colormath.color_conversions import convert_color
import pandas as pd
import math
import numpy as np
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from PIL import Image
from matplotlib import pyplot as plt
import copy
from sklearn.preprocessing import LabelEncoder
import pandas.core.algorithms as algos
from pandas import Series
import pickle
#from pandas import tools

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.svm import SVR
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from sklearn.linear_model import LinearRegression
from tensorflow import keras
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
# gpu 상태 확인 및 메모리 사용만큼 할당
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
import webcolors
from colormath.color_objects import LabColor, sRGBColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
from colormath import color_diff_matrix
import matplotlib.colors as mcolors

from math import sqrt
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cluster import KMeans
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 필요함수

In [ ]:
def cie94_color_difference(y_pred, y_true):
    """Calculate color difference by using CIE94 formulae

    See http://en.wikipedia.org/wiki/Color_difference or
    http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

    cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
    >>> 58.0
    cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
    >>> 0.0
    """

    L1, a1, b1 = y_true[0],y_true[1],y_true[2]
    L2, a2, b2 = y_pred[0],y_pred[1],y_pred[2]


    C1 = np.sqrt(a1**2 + b1**2)
    C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
    delta_L = L1 - L2
    delta_C = C1 - C2
    delta_a = a1 - a2
    delta_b = b1 - b2
    delta_H_square = delta_a**2 + delta_b**2 - delta_C**2
    loss_a = (np.sqrt((delta_L)**2
            + (delta_C)**2 / (1.0 + 0.045 * C1)**2
            + delta_H_square / (1.0 + 0.015 * C1)**2))
    return loss_a

In [ ]:
def color_diff(pred_TRANS,Y_TRANS):
    diff = []
    for i in Y_TRANS.index:
        pred = pred_TRANS.loc[i]
        true = Y_TRANS.loc[i]
        p_lab = LabColor(pred[0],pred[1],pred[2])
        t_lab = LabColor(true[0],true[1],true[2])
        delta = delta_e_cmc(p_lab , t_lab)
        diff.append(delta)
    diff=pd.DataFrame(diff)
    diff.index=Y_TRANS.index
    return(diff)

In [ ]:
def color_diff(pred_TRANS,Y_TRANS):
    diff = []
    for i in Y_TRANS.index:
        pred = list(pred_TRANS.loc[i])
        true = list(Y_TRANS.loc[i])
#        p_lab = LabColor(pred[0],pred[1],pred[2])
#        t_lab = LabColor(true[0],true[1],true[2])
        delta = cie94_color_difference(pred , true)
        diff.append(delta)
    diff=pd.DataFrame(diff)
    diff.index=Y_TRANS.index
    return(diff)

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
    # 첨 class 선언신 변수생성
    def __init__(self,input_units, num_unit,output_unit,lams,**kwargs,
                ):
        super(AttentionLayer, self).__init__(**kwargs)
        self.input_units = input_units
        self.num_unit = num_unit
        self.output_unit = output_unit
        self.lambdas = lams

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'input_units': self.input_units,
            'num_unit': self.num_unit,
            'output_unit': self.output_unit,
            'lambdas': self.lambdas,
        })
        return config

    def build(self, input_shape):
        self.kernel1 = self.add_weight("kernel1",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape =[self.input_units,self.num_unit])
        self.bias1 = self.add_weight("bias1",initializer = tf.zeros_initializer(),shape=[1,self.num_unit])



        self.kernel2 = self.add_weight("kernel2",
                    initializer = tf.keras.initializers.glorot_normal(),
                    regularizer = tf.keras.regularizers.l2(self.lambdas),
                    shape = [self.num_unit,self.output_unit])
        self.bias2 = self.add_weight("bias2",initializer = tf.zeros_initializer(),shape=[1,self.output_unit])



    def call(self, inputs):
        shape = [ tf.shape(inputs)[k] for k in range(len(inputs.shape))]

        shape_list = [shape[0]] + [1 for i in range(len(inputs.shape)-1)]
        kernel1_ = tf.tile(tf.expand_dims(self.kernel1,0),shape_list)
        kernel2_ = tf.tile(tf.expand_dims(self.kernel2,0),shape_list)

        h1 = tf.matmul(inputs,kernel1_) + self.bias1
        h1 = tf.nn.tanh(h1)
        out = tf.matmul(h1,kernel2_) + self.bias2
        out = tf.nn.softmax(out,axis=1)
        flat_output_total = tf.multiply(inputs,tf.expand_dims(out[:,:,0],axis=2))
        flat_output_total = tf.reduce_sum(flat_output_total,axis=1)
        for i in range(1,self.output_unit):
            flat_output= tf.multiply(inputs,tf.expand_dims(out[:,:,i],axis=2))
            flat_output = tf.reduce_sum(flat_output,axis=1)
            flat_output_total = tf.concat([flat_output_total,flat_output],axis=1)

        return flat_output_total

In [ ]:
    # CIE94 Keras Custom Loss
    def cie94(y_true, y_pred):
        """Calculate color difference by using CIE94 formulae

        See http://en.wikipedia.org/wiki/Color_difference or
        http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

        cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
        >>> 58.0
        cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
        >>> 0.0
        """

        L1, a1, b1 = y_true[:,0],y_true[:,1],y_true[:,2]
        L2, a2, b2 = y_pred[:,0],y_pred[:,1],y_pred[:,2]


        C1 = K.sqrt(K.pow(a1,2) + K.pow(b1,2))
        C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
        delta_L = L1 - L2
        delta_C = C1 - C2
        delta_a = a1 - a2
        delta_b = b1 - b2
        delta_H_square = K.pow(delta_a,2) + K.pow(delta_b,2) - K.pow(delta_C,2)
        loss_a = (K.sqrt(K.pow(delta_L,2)
                + K.pow(delta_C,2) / K.pow(1.0 + 0.045 * C1,2)
                + delta_H_square / K.pow(1.0 + 0.015 * C1,2)))
        return loss_a

# 데이터불러오기

In [ ]:
#데이터 불러오기
train=pd.read_csv("/content/CTgan_sample.csv",index_col=0)
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/플젝테스트/제공_모델_코드/기초데이터/test_data.csv',encoding='euc-kr',index_col=0)



#인덱스정보 따로 저장

# test_index=test['index']
# # 'index' 열을 별도로 저장하고, test 데이터셋에서 제거
# train_index = train['index']
# train = train.drop(columns=['index'])
test_index = test['index']
test = test.drop(columns=['index'])
#shape 확인
print('train shape : ',train.shape)
print('test shape : ',test.shape)

# #데이터 결합
# data=pd.concat([train,test],axis=0)

# #index 제거
# data=data[data.columns[1:]]
# data

train shape :  (3000, 173)
test shape :  (372, 173)


In [ ]:
train

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop,배합_염료1_Grey_prop,배합_염료2_Grey_prop,배합_염료3_Grey_prop,배합_염료2_Brown_prop,배합_염료1_Brown_prop,배합_염료3_Brown_prop,색상_L*,색상_a*,색상_b*
0,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,77.298144,31.975403,4.555252
1,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,67.421652,-1.012464,-19.809998
2,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,95.107735,22.398183,-12.505660
3,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,64.302653,9.994502,-2.867670
4,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,1,1,0,0,0,0,0,39.513798,71.191803,11.758286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,81.443703,13.075929,-21.114292
2996,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,88,Filaments,150,150,0,...,0,0,0,0,0,0,0,54.139497,28.978143,-1.616050
2997,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,1,0,0,0,0,0,34.300619,21.250453,60.080927
2998,PET 재생 섬유,300,96,Semi Dull,Draw Textured Yarn,96,Filaments,150,150,0,...,0,0,0,0,0,0,0,31.436466,20.119413,26.690585


In [ ]:
# train 데이터셋의 특정 컬럼들을 소수점 두 자리로 반올림
train['색상_L*'] = train['색상_L*'].round(2)
train["색상_a*"] = train['색상_a*'].round(2)
train['색상_b*'] = train['색상_b*'].round(2)


In [ ]:
# #전체 변수 보기
# np.array(data.columns)

In [ ]:
# len(np.array(data.columns))

# 범주형 및 수치형 인코딩

In [ ]:
#수치형 변수 담기
num_feature_col=[ '소재_경사정보_V2',
 '소재_경사정보_V3',
 '소재_경사정보_V6',
 '소재_경사정보_V8',
 '소재_경사정보_V9',
 '소재_경사정보_V10',
 '소재_경사정보_V11',
 '소재_경사정보_V12',
 '소재_경사정보_V13',
 '소재_경사정보_V14',
 '소재_경사정보_V15',
 '소재_위사정보_V2',
 '소재_위사정보_V3',
 '소재_위사정보_V6',
 '소재_위사정보_V8',
 '소재_위사정보_V9',
 '소재_위사정보_V10',
 '소재_위사정보_V11',
 '소재_위사정보_V12',
 '소재_위사정보_V13',
 '소재_위사정보_V14',
 '소재_위사정보_V15',
 '소재_제직정보_V3',
 '소재_제직정보_V4',
 '전처리_정보_V1',
 '전처리_정보_V2',
 '전처리_정보_V3',
 '전처리_정보_V4',
 '전처리_정보_V5',
 '전처리_정보_V6',
 '전처리_정보_V7',
 '전처리_정보_V8',
 '전처리_정보_V9',
 '전처리_정보_V10',
 '전처리_정보_V11',
 '전처리_정보_V12',
 '전처리_정보_V13',
 '전처리_정보_V14',
 '전처리_정보_V15',
 '전처리_정보_V16',
 '전처리_정보_V17',
 '전처리_정보_V18',
 '전처리_정보_V19',
 '전처리_정보_V20',
 '전처리_정보_V21',
 '전처리_정보_V22',
 '전처리_정보_V23',
 '전처리_정보_V24',
 '전처리_정보_V25',
 '전처리_정보_V26',
 '전처리_정보_V27',
 '전처리_정보_V28',
 '전처리_정보_V29',
 '전처리_정보_V30',
 '전처리_정보_V31',
 '전처리_정보_V32',
 '전처리_정보_V33',
 '전처리_정보_V34',
 '배합_염료1_Yellow',
 '배합_염료1_Red',
 '배합_염료1_Blue',
 '배합_염료2_Yellow',
 '배합_염료2_Red',
 '배합_염료2_Blue',
 '배합_염료3_Yellow',
 '배합_염료3_Red',
 '배합_염료3_Blue',
 '배합_염료1_Black',
 '배합_염료2_Black',
 '배합_염료3_Black',
 '배합_염료1_Grey',
 '배합_염료2_Grey',
 '배합_염료3_Grey',
 '배합_염료2_Brown',
 '배합_염료1_Brown',
 '배합_염료3_Brown',
 '배합_조제_V1',
 '배합_조제_V2',
 '배합_조제_V3',
 '배합_조제_V4',
 '배합_조제_V5',
 '배합_조제_V6',
 '염색_정보_V1',
 '염색_정보_V2',
 '염색_정보_V3',
 '염색_정보_V4',
 '염색_정보_V5',
 '염색_정보_V6',
 '염색_정보_V7',
 '염색_정보_V8',
 '염색_정보_V9',
 '염색_정보_V10',
 '염색_정보_V11',
 '염색_정보_V12',
 '염색_정보_V13',
 '염색_정보_V14',
 '염색_정보_V15',
 '염색_정보_V16',
 '염색_정보_V17',
 '염색_정보_V18',
 '염색_정보_V19',
 '염색_정보_V20',
 '염색_정보_V21',
 '염색_정보_V22',
 '후처리_정보_V1',
 '후처리_정보_V2',
 '후처리_정보_V3',
 '후처리_정보_V4',
 '후처리_정보_V5',
 '후처리_정보_V6',
 '후처리_정보_V7',
 '후처리_정보_V8',
 '후처리_정보_V9',
 '후처리_정보_V10',
 '후처리_정보_V11',
 '후처리_정보_V12',
 '후처리_정보_V13',
 '후처리_정보_V14',
 '후처리_정보_V15',
 '후처리_정보_V16',
 '후처리_정보_V17',
 '후처리_정보_V18',
 '후처리_정보_V19',
 '후처리_정보_V20',
 '후처리_정보_V21',
 '후처리_정보_V22',
 '후처리_정보_V23',
 '후처리_정보_V24',
 '후처리_정보_V25',
 '후처리_정보_V26',
 '후처리_정보_V27',
 '후처리_정보_V28',
 '후처리_정보_V29',
 '후처리_정보_V30',
 '후가공_정보_V1',
 '후가공_정보_V2',
 '후가공_정보_V3',
 '후가공_정보_V4',
 '후가공_정보_V5',
 '후가공_정보_V6',
 '배합_염료1_Yellow_prop',
 '배합_염료1_Red_prop',
 '배합_염료1_Blue_prop',
 '배합_염료2_Yellow_prop',
 '배합_염료2_Red_prop',
 '배합_염료2_Blue_prop',
 '배합_염료3_Yellow_prop',
 '배합_염료3_Red_prop',
 '배합_염료3_Blue_prop',
 '배합_염료1_Black_prop',
 '배합_염료2_Black_prop',
 '배합_염료3_Black_prop',
 '배합_염료1_Grey_prop',
 '배합_염료2_Grey_prop',
 '배합_염료3_Grey_prop',
 '배합_염료2_Brown_prop',
 '배합_염료1_Brown_prop',
 '배합_염료3_Brown_prop',
 '색상_L*',
 '색상_a*',
 '색상_b*'
                 ]

#범주형 변수 담기
cate_feature_col=['소재_경사정보_V1',
 '소재_경사정보_V4',
 '소재_경사정보_V5',
 '소재_경사정보_V7',
 '소재_경사정보_V16',
 '소재_위사정보_V1',
 '소재_위사정보_V4',
 '소재_위사정보_V5',
 '소재_위사정보_V7',
 '소재_위사정보_V16',
 '소재_제직정보_V1',
 '소재_제직정보_V2'
                 ]

In [ ]:
# #변수 갯수
# len(num_feature_col)+len(cate_feature_col)

In [ ]:
# # Category feature 숫자로 인코딩
# categorical_names = {}
# for feature in cate_feature_col:
#     le = LabelEncoder()
#     le.fit(data[feature])
#     data[feature+"_enc"] = le.transform(data[feature])
#     categorical_names[feature] = le.classes_

In [ ]:
# Category feature 숫자로 인코딩
categorical_names = {}
for feature in cate_feature_col:
    le = LabelEncoder()
    le.fit(test[feature])
    train[feature] = le.transform(train[feature])
    categorical_names[feature] = le.classes_

In [ ]:
# Category feature 숫자로 인코딩
categorical_names = {}
for feature in cate_feature_col:
    le = LabelEncoder()
    le.fit(test[feature])
    test[feature] = le.transform(test[feature])
    categorical_names[feature] = le.classes_

In [ ]:
# categorical_names

In [ ]:
#min max 정규화
std_model = {}
for feature in num_feature_col:
    std = MinMaxScaler()
    print(feature)
    std.fit(train[[feature]])
    train[feature+"_std"] = std.transform(train[[feature]])
    std_model[feature] = std

소재_경사정보_V2
소재_경사정보_V3
소재_경사정보_V6
소재_경사정보_V8
소재_경사정보_V9
소재_경사정보_V10
소재_경사정보_V11
소재_경사정보_V12
소재_경사정보_V13
소재_경사정보_V14
소재_경사정보_V15
소재_위사정보_V2
소재_위사정보_V3
소재_위사정보_V6
소재_위사정보_V8
소재_위사정보_V9
소재_위사정보_V10
소재_위사정보_V11
소재_위사정보_V12
소재_위사정보_V13
소재_위사정보_V14
소재_위사정보_V15
소재_제직정보_V3
소재_제직정보_V4
전처리_정보_V1
전처리_정보_V2
전처리_정보_V3
전처리_정보_V4
전처리_정보_V5
전처리_정보_V6
전처리_정보_V7
전처리_정보_V8
전처리_정보_V9
전처리_정보_V10
전처리_정보_V11
전처리_정보_V12
전처리_정보_V13
전처리_정보_V14
전처리_정보_V15
전처리_정보_V16
전처리_정보_V17
전처리_정보_V18
전처리_정보_V19
전처리_정보_V20
전처리_정보_V21
전처리_정보_V22
전처리_정보_V23
전처리_정보_V24
전처리_정보_V25
전처리_정보_V26
전처리_정보_V27
전처리_정보_V28
전처리_정보_V29
전처리_정보_V30
전처리_정보_V31
전처리_정보_V32
전처리_정보_V33
전처리_정보_V34
배합_염료1_Yellow
배합_염료1_Red
배합_염료1_Blue
배합_염료2_Yellow
배합_염료2_Red
배합_염료2_Blue
배합_염료3_Yellow
배합_염료3_Red
배합_염료3_Blue
배합_염료1_Black
배합_염료2_Black
배합_염료3_Black
배합_염료1_Grey
배합_염료2_Grey
배합_염료3_Grey
배합_염료2_Brown
배합_염료1_Brown
배합_염료3_Brown
배합_조제_V1
배합_조제_V2
배합_조제_V3
배합_조제_V4
배합_조제_V5
배합_조제_V6
염색_정보_V1
염색_정보_V2
염색_정보_V3
염색_정보_V4
염색_정보_V5
염색_정보_V6
염색_정보_V7
염색_정보_V8
염색_정보_V9

In [ ]:
#min max 정규화
std_model = {}
for feature in num_feature_col:
    std = MinMaxScaler()
    print(feature)
    std.fit(test[[feature]])
    test[feature+"_std"] = std.transform(test[[feature]])
    std_model[feature] = std

소재_경사정보_V2
소재_경사정보_V3
소재_경사정보_V6
소재_경사정보_V8
소재_경사정보_V9
소재_경사정보_V10
소재_경사정보_V11
소재_경사정보_V12
소재_경사정보_V13
소재_경사정보_V14
소재_경사정보_V15
소재_위사정보_V2
소재_위사정보_V3
소재_위사정보_V6
소재_위사정보_V8
소재_위사정보_V9
소재_위사정보_V10
소재_위사정보_V11
소재_위사정보_V12
소재_위사정보_V13
소재_위사정보_V14
소재_위사정보_V15
소재_제직정보_V3
소재_제직정보_V4
전처리_정보_V1
전처리_정보_V2
전처리_정보_V3
전처리_정보_V4
전처리_정보_V5
전처리_정보_V6
전처리_정보_V7
전처리_정보_V8
전처리_정보_V9
전처리_정보_V10
전처리_정보_V11
전처리_정보_V12
전처리_정보_V13
전처리_정보_V14
전처리_정보_V15
전처리_정보_V16
전처리_정보_V17
전처리_정보_V18
전처리_정보_V19
전처리_정보_V20
전처리_정보_V21
전처리_정보_V22
전처리_정보_V23
전처리_정보_V24
전처리_정보_V25
전처리_정보_V26
전처리_정보_V27
전처리_정보_V28
전처리_정보_V29
전처리_정보_V30
전처리_정보_V31
전처리_정보_V32
전처리_정보_V33
전처리_정보_V34
배합_염료1_Yellow
배합_염료1_Red
배합_염료1_Blue
배합_염료2_Yellow
배합_염료2_Red
배합_염료2_Blue
배합_염료3_Yellow
배합_염료3_Red
배합_염료3_Blue
배합_염료1_Black
배합_염료2_Black
배합_염료3_Black
배합_염료1_Grey
배합_염료2_Grey
배합_염료3_Grey
배합_염료2_Brown
배합_염료1_Brown
배합_염료3_Brown
배합_조제_V1
배합_조제_V2
배합_조제_V3
배합_조제_V4
배합_조제_V5
배합_조제_V6
염색_정보_V1
염색_정보_V2
염색_정보_V3
염색_정보_V4
염색_정보_V5
염색_정보_V6
염색_정보_V7
염색_정보_V8
염색_정보_V9

In [ ]:
train

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop_std,배합_염료1_Grey_prop_std,배합_염료2_Grey_prop_std,배합_염료3_Grey_prop_std,배합_염료2_Brown_prop_std,배합_염료1_Brown_prop_std,배합_염료3_Brown_prop_std,색상_L*_std,색상_a*_std,색상_b*_std
0,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.686084,0.566735,0.322884
1,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.578506,0.228029,0.156808
2,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.880009,0.468378,0.206556
3,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.544534,0.340965,0.272250
4,0,300,96,0,0,88,0,150,150,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.274608,0.969302,0.371950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.731163,0.372690,0.147949
2996,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433907,0.535934,0.280769
2997,0,300,96,0,0,96,0,150,150,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.217879,0.456571,0.701240
2998,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.186738,0.444969,0.473695


In [ ]:
test

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop_std,배합_염료1_Grey_prop_std,배합_염료2_Grey_prop_std,배합_염료3_Grey_prop_std,배합_염료2_Brown_prop_std,배합_염료1_Brown_prop_std,배합_염료3_Brown_prop_std,색상_L*_std,색상_a*_std,색상_b*_std
3719,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.807145,0.457208,0.919145
2770,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.377718,0.987935,0.442675
3306,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.373821,0.315059,0.077870
1674,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.232221,0.327342,0.061131
2383,0,300,96,0,0,96,0,150,150,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.729569,0.206463,0.238549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1094,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.504364,0.333528,0.342712
1895,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.654780,0.763160,0.451791
1020,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.235672,0.000000,0.275128
352,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.385412,0.026407,0.475452


In [ ]:
# 독립변수 X 설정
xxx1 = np.where(train.columns == '소재_경사정보_V1')[0][0]
xxx2 = np.where(train.columns == '색상_L*_std')[0][0]
X_train = train[train.columns[xxx1:xxx2]]
X_test = test[test.columns[xxx1:xxx2]]

# 종속변수 Y 설정
yyy1 = np.where(train.columns == '색상_L*')[0][0]
yyy2 = np.where(train.columns == '색상_b*')[0][0]
y_train = train[train.columns[yyy1:(yyy2+1)]]
y_test = test[test.columns[yyy1:(yyy2+1)]]


In [ ]:
# #독립변수 X에 담기
# xxx1=np.where(train.columns=='소재_경사정보_V1')[0][0]
# xxx2=np.where(train.columns=='색상_L*')[0][0]
# X=train[train.columns[xxx1:(xxx2)]]
# # #X=pd.concat([X,data_빅웨이브[['cor0_enc']]],axis=1)
# # X.index=data.index
# # X

In [ ]:
train["소재_경사정보_V1"]

0       0
1       0
2       0
3       0
4       0
       ..
2995    0
2996    0
2997    0
2998    0
2999    0
Name: 소재_경사정보_V1, Length: 3000, dtype: int64

In [ ]:
# #종속변수 Y에 담기
# yyy1=np.where(train.columns=='색상_L*')[0][0]
# yyy2=np.where(train.columns=='색상_b*')[0][0]
# Y=train[train.columns[yyy1:(yyy2+1)]]
# Y.index=train.index
# Y

In [ ]:
train

,소재_경사정보_V1,소재_경사정보_V2,소재_경사정보_V3,소재_경사정보_V4,소재_경사정보_V5,소재_경사정보_V6,소재_경사정보_V7,소재_경사정보_V8,소재_경사정보_V9,소재_경사정보_V10,...,배합_염료3_Black_prop_std,배합_염료1_Grey_prop_std,배합_염료2_Grey_prop_std,배합_염료3_Grey_prop_std,배합_염료2_Brown_prop_std,배합_염료1_Brown_prop_std,배합_염료3_Brown_prop_std,색상_L*_std,색상_a*_std,색상_b*_std
0,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.686084,0.566735,0.322884
1,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.578506,0.228029,0.156808
2,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.880009,0.468378,0.206556
3,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.544534,0.340965,0.272250
4,0,300,96,0,0,88,0,150,150,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.274608,0.969302,0.371950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.731163,0.372690,0.147949
2996,0,300,96,0,0,88,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433907,0.535934,0.280769
2997,0,300,96,0,0,96,0,150,150,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.217879,0.456571,0.701240
2998,0,300,96,0,0,96,0,150,150,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.186738,0.444969,0.473695


# LSTM 모델 설계

In [ ]:
num_input = keras.layers.Input((len(num_feature_col)-3,))
num_input

<KerasTensor: shape=(None, 158) dtype=float32 (created by layer 'input_3')>

In [ ]:
# 범주형 특성 선택
X_train_cate_input = X_train[cate_feature_col].values
X_test_cate_input = X_test[cate_feature_col].values

# 수치형 특성 선택 (마지막 3개 특성 제외)
X_train_num_input = X_train_num_input = X_train[[j+"_std" for j in num_feature_col[:-3]]].values
X_test_num_input = X_test[[j+"_std" for j in num_feature_col[:-3]]].values
# 종속변수 선택
Y_train_input = y_train.values
Y_test_input = y_test.values

# TensorFlow 데이터셋 생성
train_data2 = tf.data.Dataset.from_tensor_slices(((X_train_cate_input, X_train_num_input), Y_train_input)).batch(25).shuffle(len(X_train_cate_input))
test_data2 = tf.data.Dataset.from_tensor_slices(((X_test_cate_input, X_test_num_input), Y_test_input)).batch(25)


In [ ]:
#LSTM 모델 설계
def model_build():
    keras.backend.clear_session()

    ##범주형변수, 수치형 변수 갯수 만큼 설정
    cate_input = keras.layers.Input((12,))
    num_input = keras.layers.Input((158,))

    #카테고리변수설정(모두 소재)
    emb1 = keras.layers.Embedding(1,8)(cate_input[:,0])
    emb2 = keras.layers.Embedding(1,8)(cate_input[:,1])
    emb3 = keras.layers.Embedding(1,8)(cate_input[:,2])
    emb4 = keras.layers.Embedding(1,8)(cate_input[:,3])
    emb5 = keras.layers.Embedding(1,8)(cate_input[:,4])
    emb6 = keras.layers.Embedding(1,8)(cate_input[:,5])
    emb7 = keras.layers.Embedding(1,8)(cate_input[:,6])
    emb8 = keras.layers.Embedding(1,8)(cate_input[:,7])
    emb9 = keras.layers.Embedding(1,8)(cate_input[:,8])
    emb10 = keras.layers.Embedding(1,8)(cate_input[:,9])
    emb11 = keras.layers.Embedding(1,8)(cate_input[:,10])
    emb12 = keras.layers.Embedding(1,8)(cate_input[:,11])

    #소재 부
    part1_layer = keras.layers.Concatenate()([num_input[:,0:24],
                                              emb1,emb2,emb3,emb4,
                                              emb5,emb6,emb7,emb8,
                                              emb9,emb10,emb11,emb12
                                             ])
    part1_layer = keras.layers.Dense(96,activation="selu")(part1_layer)
    part1_layer = keras.layers.Dropout(0.3)(part1_layer)
    part1_layer = tf.expand_dims(part1_layer,axis=1)

    # 전처리_1차
    part2_layer = keras.layers.Dense(24,activation="selu")(num_input[:,24:52])
    part2_layer = keras.layers.Dense(96,activation="selu")(part2_layer)
    part2_layer = keras.layers.Dropout(0.3)(part2_layer)
    part2_layer = tf.expand_dims(part2_layer,axis=1)

    # 전처리_2차
    part3_layer = keras.layers.Dense(24,activation="selu")(num_input[:,52:58])
    part3_layer = keras.layers.Dense(96,activation="selu")(part3_layer)
    part3_layer = keras.layers.Dropout(0.3)(part3_layer)
    part3_layer = tf.expand_dims(part3_layer,axis=1)

    # 배합부
    part4_layer = keras.layers.Concatenate()([num_input[:,58:82],num_input[:,140:158]])
    part4_layer = keras.layers.Dense(96,activation="selu")(part4_layer)
    part4_layer = keras.layers.Dropout(0.3)(part4_layer)
    part4_layer = tf.expand_dims(part4_layer,axis=1)

    # 염색부
    part5_layer = keras.layers.Dense(24,activation="selu")(num_input[:,82:104])
    part5_layer = keras.layers.Dense(96,activation="selu")(part5_layer)
    part5_layer = keras.layers.Dropout(0.3)(part5_layer)
    part5_layer = tf.expand_dims(part5_layer,axis=1)

    #후처리부
    part6_layer = keras.layers.Dense(24,activation="selu")(num_input[:,104:134])
    part6_layer = keras.layers.Dense(96,activation="selu")(part6_layer)
    part6_layer = keras.layers.Dropout(0.3)(part6_layer)
    part6_layer = tf.expand_dims(part6_layer,axis=1)

    #후가공_1차 공정
    part7_layer = keras.layers.Dense(24,activation="selu")(num_input[:,134:136])
    part7_layer = keras.layers.Dense(96,activation="selu")(part7_layer)
    part7_layer = keras.layers.Dropout(0.3)(part7_layer)
    part7_layer = tf.expand_dims(part7_layer,axis=1)

    #후가공_2차 공정
    part8_layer = keras.layers.Dense(24,activation="selu")(num_input[:,136:140])
    part8_layer = keras.layers.Dense(96,activation="selu")(part8_layer)
    part8_layer = keras.layers.Dropout(0.3)(part8_layer)
    part8_layer = tf.expand_dims(part8_layer,axis=1)

    # Concatenate
    total_concat_be = keras.layers.Concatenate(axis=1)([part1_layer,
                                                        part2_layer,
                                                        part3_layer,
                                                        part4_layer,
                                                        part5_layer,
                                                        part6_layer,
                                                        part7_layer,
                                                        part8_layer])
    total_concat_be2  = AttentionLayer(96,96,2,0.01)(total_concat_be)

    # LSTM Layer
    total_concat = keras.layers.LSTM(96)(total_concat_be)

    # Dense Layer
    total_concat = keras.layers.Concatenate()([total_concat,num_input,total_concat_be2])
    total_concat = keras.layers.Dense(96,activation="selu")(total_concat)

    output = keras.layers.Dense(3,name="l_loss")(total_concat)

    # CIE94 Keras Custom Loss 함수 설계
    def cie94(y_true, y_pred):
        """Calculate color difference by using CIE94 formulae

        See http://en.wikipedia.org/wiki/Color_difference or
        http://www.brucelindbloom.com/index.html?Eqn_DeltaE_CIE94.html.

        cie94(rgb2lab((255, 255, 255)), rgb2lab((0, 0, 0)))
        >>> 58.0
        cie94(rgb2lab(rgb(0xff0000)), rgb2lab(rgb('#ff0000')))
        >>> 0.0
        """

        L1, a1, b1 = y_true[:,0],y_true[:,1],y_true[:,2]
        L2, a2, b2 = y_pred[:,0],y_pred[:,1],y_pred[:,2]


        C1 = K.sqrt(K.pow(a1,2) + K.pow(b1,2))
        C2 = K.sqrt(K.pow(a2,2) + K.pow(b2,2))
        delta_L = L1 - L2
        delta_C = C1 - C2
        delta_a = a1 - a2
        delta_b = b1 - b2
        delta_H_square = K.pow(delta_a,2) + K.pow(delta_b,2) - K.pow(delta_C,2)
        loss_a = (K.sqrt(K.pow(delta_L,2)
                + K.pow(delta_C,2) / K.pow(1.0 + 0.045 * C1,2)
                + delta_H_square / K.pow(1.0 + 0.015 * C1,2)))
        return loss_a

    model = keras.models.Model(inputs=[cate_input,num_input],outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.01),loss=[cie94], metrics=[cie94])
    return(model)

In [ ]:
model = model_build()

In [ ]:
X_test_num_input.shape

(372, 158)

In [ ]:
# # ModelCheckpoint 콜백 정의
# checkpoint_filepath = './model/MODEL.h5'  # 모델을 저장할 파일 경로
# model_checkpoint_callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_best_only=True,  # 가장 좋은 모델만 저장
#     monitor='val_cie94',  # 모니터링할 지표 (validation_data에서의 cie94 평가 지표)
#     mode='min',  # 최솟값을 찾기 위해 모니터링
#     verbose=1  # 저장할 때 로그 출력
# )
from keras import backend as K
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

# RMSE 함수 정의
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))


# 모델 컴파일, 여러 평가 지표 추가
model.compile(optimizer='adam', loss='mse', metrics=[cie94, rmse])

# ModelCheckpoint 콜백 정의
checkpoint_filepath = './model/MODEL.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_rmse',  # 모니터링할 지표 선택, 'val_cie94'로 변경 가능
    mode='min',
    verbose=1
)

# 모델 학습
tf.random.set_seed(42)
history = model.fit(
    train_data2,
    epochs=1200,
    validation_data=test_data2,
    callbacks=[model_checkpoint_callback]
)



Epoch 1/1200
118/120 [============================>.] - ETA: 0s - loss: 711.2254 - cie94: 30.2103 - rmse: 26.0301
Epoch 1: val_rmse improved from inf to 24.12887, saving model to ./model/MODEL.h5
120/120 [==============================] - 13s 27ms/step - loss: 709.4296 - cie94: 30.1242 - rmse: 26.0044 - val_loss: 587.1265 - val_cie94: 26.0556 - val_rmse: 24.1289
Epoch 2/1200
120/120 [==============================] - ETA: 0s - loss: 562.9667 - cie94: 25.6115 - rmse: 23.5909
Epoch 2: val_rmse did not improve from 24.12887
120/120 [==============================] - 2s 17ms/step - loss: 562.9667 - cie94: 25.6115 - rmse: 23.5909 - val_loss: 589.6187 - val_cie94: 26.4216 - val_rmse: 24.1766
Epoch 3/1200
119/120 [============================>.] - ETA: 0s - loss: 561.4753 - cie94: 25.6154 - rmse: 23.5574
Epoch 3: val_rmse improved from 24.12887 to 24.06353, saving model to ./model/MODEL.h5
120/120 [==============================] - 2s 18ms/step - loss: 563.3434 - cie94: 25.6467 - rmse: 23.594

In [ ]:
# #1차 학습
# tf.random.set_seed(42)
# a=model.fit(train_data2,
#             epochs=1200,
#             validation_data=test_data2,
#             callbacks=[model_checkpoint_callback])

# 훈련 데이터에 대한 RMSE 및 CIE94 결과
train_rmse = history.history['rmse']
train_cie94 = history.history['cie94']

# 검증 데이터에 대한 RMSE 및 CIE94 결과
val_rmse = history.history['val_rmse']
val_cie94 = history.history['val_cie94']

print("Training RMSE:", train_rmse)
print("Training CIE94:", train_cie94)
print("Validation RMSE:", val_rmse)
print("Validation CIE94:", val_cie94)


In [ ]:
#모델 불러오기
model.load_weights("./model/MODEL.h5")

In [ ]:
#카테고리 데이터 내보내기
#with open("./model/categorical_names_new.pkl","wb") as f:
#    pickle.dump(categorical_names,f)

# MinMaxScaler model pkl 내보내기
#with open("./model/Dytec_std_model_new.pkl", "wb") as f:
#     pickle.dump(std_model,f)

In [ ]:
#예측 데이터 생성
Y_pred=pd.DataFrame(model.predict([X_train_cate_input,X_train_num_input]))
Y_pred.columns=['색상_L*', '색상_a*', '색상_b*']
Y_pred.index=X_train_df.index

#예측데이터 재할당
pred_TRANS = Y_pred

#실제 데이터 재규격화
Y_TRANS=y_train_df
Y_TRANS.columns=[['색상_L*', '색상_a*', '색상_b*']]

#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)

#이상치 확인
train_color_diff=diff_color.sort_values(0,ascending=False)
train_color_diff=train_color_diff.loc[train_color_diff[0]>=100]
train_color_diff_index=list(train_color_diff.index)
train_color_diff_index

In [ ]:
np.mean(diff_color)

In [ ]:
#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)
diff_color

In [ ]:
#lab값을 rgb로 변경
y_test_rgb=lab_to_rgb(Y_TRANS)
y_test_rgb

In [ ]:
#lab값을 rgb로 변경
pred_rgb=lab_to_rgb(pred_TRANS)
pred_rgb

# Test 데이터 확인

In [ ]:
#예측 확인
model.predict([X_test_cate_input,X_test_num_input])[:5]

#예측 데이터 생성
Y_pred=pd.DataFrame(model.predict([X_test_cate_input,X_test_num_input]))
Y_pred.columns=['색상_L*', '색상_a*', '색상_b*']
Y_pred.index=X_test_df.index

#예측데이터 재할당
pred_TRANS = Y_pred
pred_TRANS

#실제 데이터 재규격화
Y_TRANS=y_test_df
Y_TRANS.columns=[['색상_L*', '색상_a*', '색상_b*']]
Y_TRANS

#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)
diff_color

#이상치 확인
test_color_diff=diff_color.sort_values(0,ascending=False)
test_color_diff= test_color_diff.loc[test_color_diff[0]>=100]
test_color_diff_index=list(test_color_diff.index)
test_color_diff_index


In [ ]:
Y_pred

In [ ]:
y_test_df

In [ ]:
#L*
r2_score(y_test_df[y_test_df.columns[0]],Y_pred[Y_pred.columns[0]])


In [ ]:
#a*
r2_score(y_test_df[y_test_df.columns[1]],Y_pred[Y_pred.columns[1]])


In [ ]:
#b*
r2_score(y_test_df[y_test_df.columns[2]],Y_pred[Y_pred.columns[2]])


In [ ]:
np.mean(diff_color)

In [ ]:
#색차 확인
diff_color=color_diff(pred_TRANS,Y_TRANS)
diff_color

In [ ]:
#lab값을 rgb로 변경
y_test_rgb=lab_to_rgb(Y_TRANS)
y_test_rgb

In [ ]:
#lab값을 rgb로 변경
pred_rgb=lab_to_rgb(pred_TRANS)
pred_rgb

# 그래프 그리기

In [ ]:
for i in range(len(y_test_rgb)//3+1):
    plt.figure(figsize=(15,10))
    plt.subplot(181)
    plt.imshow(Image.new("RGB", (50,50),(int(y_test_rgb['r'][y_test_rgb.index[3*i]]*255),int(y_test_rgb['g'][y_test_rgb.index[3*i]]*255),int(y_test_rgb['b'][y_test_rgb.index[3*i]]*255))))
    plt.title('index: '+str(y_test_rgb.index[3*i])+'\nReal Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(182)
    plt.imshow(Image.new("RGB", (50,50),(int(pred_rgb['r'][y_test_rgb.index[3*i]]*255),int(pred_rgb['g'][y_test_rgb.index[3*i]]*255),int(pred_rgb['b'][y_test_rgb.index[3*i]]*255))))
    plt.title('diff: '+str(round(diff_color.loc[diff_color.index[3*i]][0],3))+'\npredicted Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(184)
    plt.imshow(Image.new("RGB", (50,50),(int(y_test_rgb['r'][y_test_rgb.index[3*i+1]]*255),int(y_test_rgb['g'][y_test_rgb.index[3*i+1]]*255),int(y_test_rgb['b'][y_test_rgb.index[3*i+1]]*255))))
    plt.title('index: '+str(y_test_rgb.index[3*i+1])+'\nReal Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(185)
    plt.imshow(Image.new("RGB", (50,50),(int(pred_rgb['r'][y_test_rgb.index[3*i+1]]*255),int(pred_rgb['g'][y_test_rgb.index[3*i+1]]*255),int(pred_rgb['b'][y_test_rgb.index[3*i+1]]*255))))
    plt.title('diff: '+str(round(diff_color.loc[diff_color.index[3*i+1]][0],3))+'\npredicted Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(187)
    plt.imshow(Image.new("RGB", (50,50),(int(y_test_rgb['r'][y_test_rgb.index[3*i+2]]*255),int(y_test_rgb['g'][y_test_rgb.index[3*i+2]]*255),int(y_test_rgb['b'][y_test_rgb.index[3*i+2]]*255))))
    plt.title('index: '+str(y_test_rgb.index[3*i+2])+'\nReal Color')
    plt.xticks([])
    plt.yticks([])

    plt.subplot(188)
    plt.imshow(Image.new("RGB", (50,50),(int(pred_rgb['r'][y_test_rgb.index[3*i+2]]*255),int(pred_rgb['g'][y_test_rgb.index[3*i+2]]*255),int(pred_rgb['b'][y_test_rgb.index[3*i+2]]*255))))
    plt.title('diff: '+str(round(diff_color.loc[diff_color.index[3*i+2]][0],3))+'\npredicted Color')
    plt.xticks([])
    plt.yticks([])
    plt.show()